In [50]:
import pandas as pd
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from pandas import DataFrame as df

In [51]:
train=pd.read_csv("train.csv")
train.dropna(axis=0, how='any', inplace=True)
train=train.reset_index()
ytrain=train["Transported"]
xtrain=train.drop(["Transported","PassengerId","Name"],axis=1)

test=pd.read_csv("test.csv")
xtest=test.drop(["PassengerId","Name"],axis=1)

In [52]:
ytrain[:10]

0    False
1     True
2    False
3    False
4     True
5     True
6     True
7     True
8     True
9     True
Name: Transported, dtype: bool

In [53]:
category_list=["HomePlanet",'Destination',"Age","VIP",'cabindeck','cabinside']
num_list=["Age",'RoomService','FoodCourt','CryoSleep','ShoppingMall','Spa','VRDeck','cabinnum']

In [54]:
def minus_mean(list1):
    print(list1)
    list1=pd.DataFrame(list1)
    mean3=list1.mean(axis=0)
    print(mean3)
    list1-=mean3
    std=list1.std(axis=0)
    list1/=std
    print(list1)
    return list1

In [55]:
def boolprocess(list2):
    list3=[]
    for i in range(list2.shape[0]):
        bool1=list2[i]
        if bool1==True:
            list3.append(1)
        elif bool1==False:
            list3.append(0)
        else:
            list3.append(0.5)
    return pd.DataFrame(list3)

In [56]:
def fill(xtrain,key):
    xtrain['isna']=0
    mask=xtrain[key].isnull()  
    xtrain.loc[mask,'isna']=1
    xtrain[key].fillna(0,inplace=True)
    return xtrain[key]

In [57]:
def input_preprocess(input):
    cabin=input["Cabin"]
    cabin1=[]
    cabin2=[]
    cabin2sum=0
    cabin2num=0
    cabin3=[]
    output=pd.DataFrame([])
    for i in range(cabin.shape[0]):
        try:
            cabin1.append(cabin[i][0])
            cabin2.append(int(cabin[i][2:-2]))
            cabin3.append(cabin[i][-1])
            cabin2sum+=int(cabin[i][2:-2])
            cabin2num+=1
        except:
            cabin1.append('F')
            cabin2.append(600)
            cabin3.append('S')
    cabin1=pd.DataFrame(cabin1,columns=["cabindeck"])
    cabin2=pd.DataFrame(cabin2,columns=["cabinnum"])
    cabin3=pd.DataFrame(cabin3,columns=["cabinside"])

    #output=pd.concat([output,cabin1,cabin2,cabin3],axis=1)
    input=pd.concat([input,cabin1,cabin2,cabin3],axis=1)

    category_list=["HomePlanet",'Destination','cabindeck','cabinside']
    num_list=["Age",'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','cabinnum']
    bool_list=["CryoSleep","VIP"]

    for i in bool_list+category_list:
        output=pd.concat([output,pd.get_dummies(input[i],prefix=i)],axis=1)

    for i in num_list:
        output=pd.concat([output,pd.DataFrame(preprocessing.scale(input[i]),columns=[i])],axis=1)
        output[i]=fill(output,i)
    output=output.drop(["isna"],axis=1)
    return output

In [58]:
for i in range(ytrain.shape[0]):
    if  ytrain[i]:
        ytrain[i]=1
    else:
        ytrain[i]=0

C:\Users\yutia\AppData\Local\Temp\ipykernel_55464\1517399558.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ytrain[i]=0


In [59]:
xtrain=input_preprocess(xtrain)

In [60]:
data=pd.concat([xtrain,ytrain],axis=1)
data.to_csv("after2.csv",index=0)